In [5]:
import os
import cv2
from scipy import interpolate
import numpy as np

In [ ]:
# Set up the paths for input and output images
BASE_PATH = os.path.abspath('.')
DATA_PATH = os.path.join(BASE_PATH, 'data')
INPUT_PATH = os.path.join(BASE_PATH, 'images/input')
OUTPUT_PATH = os.path.join(BASE_PATH, 'images/output')

# Get the name of the input image from user input
image_name = input("Enter the name of the image: ")

# Construct the paths for input and output images
INPUT_IMAGE_PATH = os.path.join(INPUT_PATH, image_name)
output_name = 'output_' + image_name
OUTPUT_IMAGE_PATH = os.path.join(OUTPUT_PATH, output_name)

In [ ]:
# Load the input image
input_image = cv2.imread(INPUT_IMAGE_PATH)

# Upsample the image using cubic B-spline interpolation
scale = 5
height, width = input_image.shape[:2]
new_height, new_width = scale * height, scale * width
fx, fy = float(new_width) / width, float(new_height) / height
new_image = cv2.resize(input_image, (new_width, new_height), interpolation=cv2.INTER_CUBIC)

# Split the image into RGB channels
r, g, b = cv2.split(new_image)

# Perform cubic B-spline interpolation on each channel
interp_r = interpolate.interp2d(np.arange(0, new_width), np.arange(0, new_height), r, kind='cubic')
interp_g = interpolate.interp2d(np.arange(0, new_width), np.arange(0, new_height), g, kind='cubic')
interp_b = interpolate.interp2d(np.arange(0, new_width), np.arange(0, new_height), b, kind='cubic')

# Create a new image by combining the interpolated RGB channels
output_image = cv2.merge((
    interp_r(np.arange(0, new_width), np.arange(0, new_height)),
    interp_g(np.arange(0, new_width), np.arange(0, new_height)),
    interp_b(np.arange(0, new_width), np.arange(0, new_height)),
))

# Convert the output image to 8-bit depth
output_image = np.clip(output_image, 0, 255).astype('uint8')

# Save the output image
cv2.imwrite(OUTPUT_IMAGE_PATH, output_image)